In [ ]:
import pickle
import numpy as np
import pandas as pd

## Function

In [211]:
def get_performance_metrics(true, estimate_file, dgp_file, bootstrap_iterations=200, g_true_index = -1):
    with open(estimate_file, "rb") as f:
        with open(dgp_file, "rb") as g:
            data = pickle.load(g)
            estimates = pickle.load(f)
            results = []
            for i, (e, d) in enumerate(zip(estimates, data)):
                if e is None:
                    continue
                beta = e["params"]["beta"]
                g = e["params"]["g"]
                beta_analytical_se = e["analytical_se"]["beta"]
                conf_beta = e["analytical_conf_interval"]["beta"]

                bootstrap_se = e.get("bootstrap_se")
                beta_bootstrap_se = bootstrap_se.get("beta") if isinstance(bootstrap_se, dict) else 0

                bootstrap_conf = e.get("bootstrap_conf_interval")
                conf_beta_bootstrap = bootstrap_conf.get("beta") if isinstance(bootstrap_conf, dict) else 0

                fit_duration = e["fit_duration"]

                # Get same g
                g_true = d[g_true_index]
                g_est_size = len(g_true)
                g_est = np.empty(g_est_size, dtype=np.int8)
                for label, inds in g.items():
                    g_est[inds] = label

                # Get perc correct
                perc_correct = np.mean(g_est == g_true)

                # Get bias
                bias = np.mean(beta - true)

                # Get RMSE
                mse = np.mean((beta - true) ** 2)

                # Get confidence size
                conf_size = (conf_beta[1] - conf_beta[0]).mean()
                conf_size_bootstrap = 0

                has_bootstrap = bootstrap_se is not None and bootstrap_se != 0
                if has_bootstrap:
                    fit_duration /= bootstrap_iterations

                if conf_beta_bootstrap is not None and conf_beta_bootstrap != 0:
                    conf_size_bootstrap = (conf_beta_bootstrap[1] - conf_beta_bootstrap[0]).mean()

                # Get coverage
                coverage = np.mean((true >= conf_beta[0]) & (true <= conf_beta[1]))
                coverage_bootstrap = 0
                if conf_beta_bootstrap is not None and conf_beta_bootstrap != 0:
                    coverage_bootstrap = np.mean((true >= conf_beta_bootstrap[0]) & (true <= conf_beta_bootstrap[1]))

                # Get mean se
                mean_se = np.mean(beta_analytical_se)
                mean_se_bootstrap = np.mean(beta_bootstrap_se)

                results.append({
                    "perc_correct": perc_correct,
                    "bias": bias,
                    "mse": mse,
                    "conf_size": conf_size,
                    "conf_size_bootstrap": conf_size_bootstrap,
                    "coverage": coverage,
                    "coverage_bootstrap": coverage_bootstrap,
                    "mean_se": mean_se,
                    "mean_se_bootstrap": mean_se_bootstrap,
                    "fit_duration": fit_duration
                })

            results_df = pd.DataFrame(results)
            results_means = results_df.mean()
            results_means["rmse"] = np.sqrt(results_means["mse"])
            return results_means.to_dict()

# DGP1 (g=3)

In [213]:
results = []
for n in (100, 200):
    for t in (20, 50):
        for g in (3, 6):
            for k in (3,):
                estimate_file = f"estimates/dgp1_n{n}_t{t}_G{g}_k{k}_full.pkl"
                dgp_file = f"development/generated_data/dgp1_n{n}_t{t}_G{g}_k{k}.pkl"
                true = np.array([[i, i, i] for i in range(1, g + 1)])

                metrics = get_performance_metrics(true, estimate_file, dgp_file)

                # Convert dict to DataFrame (assume keys are metric names, values are scalars or 1D)
                row = dict(metrics)  # in case it's already a dict
                row["n"] = n
                row["t"] = t
                row["G"] = g
                row["k"] = k

                results.append(row)

# Now convert the list of dicts into a proper DataFrame
final_results = pd.DataFrame(results)

# Optional: reorder columns
cols = ["n", "t", "G", "k"] + [col for col in final_results.columns if col not in ["n", "t", "G", "k"]]
final_results = final_results[cols]
print(final_results.round(3).sort_values(by="G").to_latex(float_format="%.3f", index=False, escape=False, column_format="l" + "r" * (len(cols) - 1)))
print(final_results[["G", "rmse"]].round(3).sort_values(by="G")[["rmse"]].to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
n & t & G & k & perc_correct & bias & mse & conf_size & conf_size_bootstrap & coverage & coverage_bootstrap & mean_se & mean_se_bootstrap & fit_duration & rmse \\
\midrule
100 & 20 & 3 & 3 & 1.000 & 0.000 & 0.002 & 0.149 & 0.155 & 0.931 & 0.938 & 0.038 & 0.039 & 0.053 & 0.040 \\
100 & 50 & 3 & 3 & 1.000 & -0.000 & 0.001 & 0.094 & 0.098 & 0.932 & 0.942 & 0.024 & 0.025 & 0.119 & 0.025 \\
200 & 20 & 3 & 3 & 1.000 & -0.000 & 0.001 & 0.106 & 0.108 & 0.942 & 0.944 & 0.027 & 0.028 & 0.082 & 0.028 \\
200 & 50 & 3 & 3 & 1.000 & -0.000 & 0.000 & 0.067 & 0.068 & 0.943 & 0.944 & 0.017 & 0.017 & 0.217 & 0.018 \\
100 & 20 & 6 & 3 & 0.999 & -0.002 & 0.004 & 0.207 & 0.000 & 0.906 & 0.000 & 0.053 & 0.000 & 2.329 & 0.067 \\
100 & 50 & 6 & 3 & 0.985 & 0.005 & 0.014 & 0.133 & 0.000 & 0.900 & 0.000 & 0.034 & 0.000 & 4.516 & 0.120 \\
200 & 20 & 6 & 3 & 1.000 & 0.001 & 0.002 & 0.149 & 0.000 & 0.931 & 0.000 & 0.038 & 0.000 & 4.313 & 0.040 \\
200 & 50 & 6 & 3 & 0.995 &

## DGP 2

In [215]:
results = []
for n in (100, 200):
    for t in (20, 50):
        for g in (3, 6):
            for k in (3,):
                estimate_file = f"estimates/dgp2_n{n}_t{t}_G{g}_k{k}_full.pkl"
                dgp_file = f"development/generated_data/dgp2_n{n}_t{t}_G{g}_k{k}.pkl"
                true = np.array([[i, i, i] for i in range(1, g + 1)])

                metrics = get_performance_metrics(true, estimate_file, dgp_file, g_true_index=3)

                # Convert dict to DataFrame (assume keys are metric names, values are scalars or 1D)
                row = dict(metrics)  # in case it's already a dict
                row["n"] = n
                row["t"] = t
                row["G"] = g
                row["k"] = k

                results.append(row)

# Now convert the list of dicts into a proper DataFrame
final_results = pd.DataFrame(results)

# Optional: reorder columns
cols = ["n", "t", "G", "k"] + [col for col in final_results.columns if col not in ["n", "t", "G", "k"]]
final_results = final_results[cols]
print(final_results.round(3).sort_values(by="G").to_latex(float_format="%.3f", index=False, escape=False, column_format="l" + "r" * (len(cols) - 1)))
print(final_results[["G", "rmse"]].round(3).sort_values(by="G")[["rmse"]].to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
n & t & G & k & perc_correct & bias & mse & conf_size & conf_size_bootstrap & coverage & coverage_bootstrap & mean_se & mean_se_bootstrap & fit_duration & rmse \\
\midrule
100 & 20 & 3 & 3 & 1.000 & -0.001 & 0.005 & 0.238 & 0.282 & 0.922 & 0.947 & 0.061 & 0.072 & 0.268 & 0.073 \\
100 & 50 & 3 & 3 & 1.000 & 0.000 & 0.001 & 0.124 & 0.131 & 0.934 & 0.945 & 0.032 & 0.033 & 0.372 & 0.035 \\
200 & 20 & 3 & 3 & 1.000 & 0.001 & 0.002 & 0.166 & 0.178 & 0.939 & 0.951 & 0.042 & 0.045 & 0.306 & 0.048 \\
200 & 50 & 3 & 3 & 1.000 & -0.001 & 0.001 & 0.087 & 0.090 & 0.936 & 0.944 & 0.022 & 0.023 & 0.705 & 0.024 \\
100 & 20 & 6 & 3 & 0.995 & 0.005 & 0.016 & 0.341 & 0.000 & 0.903 & 0.000 & 0.087 & 0.000 & 7.200 & 0.125 \\
100 & 50 & 6 & 3 & 0.999 & 0.002 & 0.004 & 0.176 & 0.000 & 0.917 & 0.000 & 0.045 & 0.000 & 9.083 & 0.063 \\
200 & 20 & 6 & 3 & 1.000 & -0.000 & 0.005 & 0.240 & 0.000 & 0.925 & 0.000 & 0.061 & 0.000 & 7.484 & 0.071 \\
200 & 50 & 6 & 3 & 1.000 & 

## DGP2 (SJ)

In [124]:
results = []
for n in (100, 200):
    for t in (20, 50):
        for g in (3, 6):
            for k in (3,):
                estimate_file = f"estimates/dgp2_n{n}_t{t}_G{g}_k{k}_su_ju.pkl"
                dgp_file = f"development/generated_data/dgp2_n{n}_t{t}_G{g}_k{k}.pkl"
                true = np.array([[i, i, i] for i in range(1, g + 1)])

                metrics = get_performance_metrics(true, estimate_file, dgp_file, g_true_index=3)

                # Convert dict to DataFrame (assume keys are metric names, values are scalars or 1D)
                row = dict(metrics)  # in case it's already a dict
                row["n"] = n
                row["t"] = t
                row["G"] = g
                row["k"] = k

                results.append(row)

# Now convert the list of dicts into a proper DataFrame
final_results = pd.DataFrame(results)

# Optional: reorder columns
cols = ["n", "t", "G", "k"] + [col for col in final_results.columns if col not in ["n", "t", "G", "k"]]
final_results = final_results[cols]
print(
    final_results.round(3)
    .sort_values(by="G")
    .to_latex(float_format="%.3f", index=False, escape=False, column_format="l" + "r" * (len(cols) - 1))
)
print(final_results[["G", "rmse"]].round(3).sort_values(by="G")[["rmse"]].to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
n & t & G & k & perc_correct & bias & mse & conf_size & conf_size_bootstrap & coverage & coverage_bootstrap & mean_se & mean_se_bootstrap & fit_duration & rmse \\
\midrule
100 & 20 & 3 & 3 & 0.996 & -0.002 & 0.004 & 0.176 & 0.000 & 0.816 & 0.000 & 0.045 & 0.000 & 0.981 & 0.065 \\
100 & 50 & 3 & 3 & 1.000 & -0.001 & 0.001 & 0.102 & 0.000 & 0.899 & 0.000 & 0.026 & 0.000 & 2.795 & 0.031 \\
200 & 20 & 3 & 3 & 0.997 & 0.001 & 0.003 & 0.122 & 0.000 & 0.761 & 0.000 & 0.031 & 0.000 & 2.421 & 0.057 \\
200 & 50 & 3 & 3 & 1.000 & -0.000 & 0.001 & 0.071 & 0.000 & 0.861 & 0.000 & 0.018 & 0.000 & 6.649 & 0.024 \\
100 & 20 & 6 & 3 & 0.942 & 0.003 & 0.064 & 0.345 & 0.000 & 0.630 & 0.000 & 0.088 & 0.000 & 1.896 & 0.253 \\
100 & 50 & 6 & 3 & 0.981 & 0.001 & 0.032 & 0.192 & 0.000 & 0.712 & 0.000 & 0.049 & 0.000 & 5.286 & 0.178 \\
200 & 20 & 6 & 3 & 0.963 & -0.001 & 0.040 & 0.230 & 0.000 & 0.539 & 0.000 & 0.059 & 0.000 & 5.272 & 0.201 \\
200 & 50 & 6 & 3 & 0.996 &

In [125]:
results = []
for n in (100, 200):
    for t in (20, 50):
        for g in (3, 6):
            for k in (3,):
                estimate_file = f"estimates/old/dgp2_n{n}_t{t}_G{g}_k{k}_su_ju.pkl"
                dgp_file = f"development/generated_data/dgp2_n{n}_t{t}_G{g}_k{k}.pkl"
                true = np.array([[i, i, i] for i in range(1, g + 1)])

                metrics = get_performance_metrics(true, estimate_file, dgp_file, g_true_index=3)

                # Convert dict to DataFrame (assume keys are metric names, values are scalars or 1D)
                row = dict(metrics)  # in case it's already a dict
                row["n"] = n
                row["t"] = t
                row["G"] = g
                row["k"] = k

                results.append(row)

# Now convert the list of dicts into a proper DataFrame
final_results = pd.DataFrame(results)

# Optional: reorder columns
cols = ["n", "t", "G", "k"] + [col for col in final_results.columns if col not in ["n", "t", "G", "k"]]
final_results = final_results[cols]
print(
    final_results.round(3)
    .sort_values(by="G")
    .to_latex(float_format="%.3f", index=False, escape=False, column_format="l" + "r" * (len(cols) - 1))
)
print(final_results[["G", "rmse"]].round(3).sort_values(by="G")[["rmse"]].to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
n & t & G & k & perc_correct & bias & mse & conf_size & conf_size_bootstrap & coverage & coverage_bootstrap & mean_se & mean_se_bootstrap & fit_duration & rmse \\
\midrule
100 & 20 & 3 & 3 & 0.995 & -0.003 & 0.007 & 0.023 & 0.285 & 0.124 & 0.939 & 0.006 & 0.073 & 0.427 & 0.086 \\
100 & 50 & 3 & 3 & 1.000 & -0.001 & 0.001 & 0.017 & 0.133 & 0.198 & 0.956 & 0.004 & 0.034 & 0.929 & 0.031 \\
200 & 20 & 3 & 3 & 0.997 & 0.001 & 0.003 & 0.012 & 0.163 & 0.083 & 0.867 & 0.003 & 0.041 & 1.138 & 0.051 \\
200 & 50 & 3 & 3 & 1.000 & -0.001 & 0.001 & 0.009 & 0.088 & 0.139 & 0.920 & 0.002 & 0.022 & 2.609 & 0.024 \\
100 & 20 & 6 & 3 & 0.945 & 0.006 & 0.061 & 0.045 & 0.000 & 0.113 & 0.000 & 0.011 & 0.000 & 1.902 & 0.247 \\
100 & 50 & 6 & 3 & 0.977 & 0.009 & 0.039 & 0.035 & 0.000 & 0.204 & 0.000 & 0.009 & 0.000 & 5.349 & 0.196 \\
200 & 20 & 6 & 3 & 0.965 & 0.001 & 0.037 & 0.023 & 0.000 & 0.075 & 0.000 & 0.006 & 0.000 & 5.299 & 0.192 \\
200 & 50 & 6 & 3 & 0.999 & 

## DGP3

In [126]:
results = []
for n in (100, 200):
    for t in (20, 50):
        for g in (3, 6):
            for k in (3,):
                estimate_file = f"estimates/dgp3_n{n}_t{t}_G{g}_k{k}.pkl"
                dgp_file = f"development/generated_data/dgp3_n{n}_t{t}_G{g}_k{k}.pkl"
                true = np.array([[i, i, i] for i in range(1, g + 1)])

                metrics = get_performance_metrics(true, estimate_file, dgp_file, g_true_index=2)

                # Convert dict to DataFrame (assume keys are metric names, values are scalars or 1D)
                row = dict(metrics)  # in case it's already a dict
                row["n"] = n
                row["t"] = t
                row["G"] = g
                row["k"] = k

                results.append(row)

# Now convert the list of dicts into a proper DataFrame
final_results = pd.DataFrame(results)

# Optional: reorder columns
cols = ["n", "t", "G", "k"] + [col for col in final_results.columns if col not in ["n", "t", "G", "k"]]
final_results = final_results[cols]
print(
    final_results.round(3)
    .sort_values(by="G")
    .to_latex(float_format="%.3f", index=False, escape=False, column_format="l" + "r" * (len(cols) - 1))
)

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
n & t & G & k & perc_correct & bias & mse & conf_size & conf_size_bootstrap & coverage & coverage_bootstrap & mean_se & mean_se_bootstrap & fit_duration & rmse \\
\midrule
100 & 20 & 3 & 3 & 0.999 & -0.001 & 0.003 & 0.156 & 0.000 & 0.845 & 0.000 & 0.040 & 0.000 & 0.952 & 0.055 \\
100 & 50 & 3 & 3 & 1.000 & -0.000 & 0.001 & 0.097 & 0.000 & 0.892 & 0.000 & 0.025 & 0.000 & 2.184 & 0.030 \\
200 & 20 & 3 & 3 & 0.999 & -0.000 & 0.002 & 0.109 & 0.000 & 0.796 & 0.000 & 0.028 & 0.000 & 2.971 & 0.043 \\
200 & 50 & 3 & 3 & 1.000 & 0.001 & 0.000 & 0.068 & 0.000 & 0.877 & 0.000 & 0.017 & 0.000 & 7.189 & 0.022 \\
100 & 20 & 6 & 3 & 0.982 & -0.002 & 0.029 & 0.259 & 0.000 & 0.701 & 0.000 & 0.066 & 0.000 & 2.167 & 0.169 \\
100 & 50 & 6 & 3 & 0.999 & 0.002 & 0.005 & 0.149 & 0.000 & 0.794 & 0.000 & 0.038 & 0.000 & 4.526 & 0.073 \\
200 & 20 & 6 & 3 & 0.991 & -0.008 & 0.017 & 0.175 & 0.000 & 0.573 & 0.000 & 0.045 & 0.000 & 9.444 & 0.130 \\
200 & 50 & 6 & 3 & 1.000 

In [127]:
results = []
for n in (100, 200):
    for t in (20, 50):
        for g in (3, 6):
            for k in (3,):
                estimate_file = f"estimates/old/dgp3_n{n}_t{t}_G{g}_k{k}.pkl"
                dgp_file = f"development/generated_data/dgp3_n{n}_t{t}_G{g}_k{k}.pkl"
                true = np.array([[i, i, i] for i in range(1, g + 1)])

                metrics = get_performance_metrics(true, estimate_file, dgp_file, g_true_index=2)

                # Convert dict to DataFrame (assume keys are metric names, values are scalars or 1D)
                row = dict(metrics)  # in case it's already a dict
                row["n"] = n
                row["t"] = t
                row["G"] = g
                row["k"] = k

                results.append(row)

# Now convert the list of dicts into a proper DataFrame
final_results = pd.DataFrame(results)

# Optional: reorder columns
cols = ["n", "t", "G", "k"] + [col for col in final_results.columns if col not in ["n", "t", "G", "k"]]
final_results = final_results[cols]
print(
    final_results.round(3)
    .sort_values(by="G")
    .to_latex(float_format="%.3f", index=False, escape=False, column_format="l" + "r" * (len(cols) - 1))
)
print(final_results[["G", "rmse"]].round(3).sort_values(by="G")[["rmse"]].to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
n & t & G & k & perc_correct & bias & mse & conf_size & conf_size_bootstrap & coverage & coverage_bootstrap & mean_se & mean_se_bootstrap & fit_duration & rmse \\
\midrule
100 & 20 & 3 & 3 & 0.999 & -0.001 & 0.003 & 0.026 & 0.196 & 0.182 & 0.916 & 0.007 & 0.050 & 0.309 & 0.055 \\
100 & 50 & 3 & 3 & 1.000 & -0.000 & 0.001 & 0.017 & 0.113 & 0.223 & 0.935 & 0.004 & 0.029 & 0.589 & 0.030 \\
200 & 20 & 3 & 3 & 0.999 & -0.000 & 0.002 & 0.013 & 0.133 & 0.110 & 0.874 & 0.003 & 0.034 & 1.159 & 0.043 \\
200 & 50 & 3 & 3 & 1.000 & 0.001 & 0.000 & 0.009 & 0.078 & 0.154 & 0.908 & 0.002 & 0.020 & 2.284 & 0.022 \\
100 & 20 & 6 & 3 & 0.985 & 0.000 & 0.026 & 0.055 & 0.000 & 0.201 & 0.000 & 0.014 & 0.000 & 2.107 & 0.163 \\
100 & 50 & 6 & 3 & 0.997 & -0.001 & 0.007 & 0.038 & 0.000 & 0.268 & 0.000 & 0.010 & 0.000 & 7.294 & 0.082 \\
200 & 20 & 6 & 3 & 0.992 & -0.008 & 0.017 & 0.028 & 0.000 & 0.126 & 0.000 & 0.007 & 0.000 & 7.512 & 0.131 \\
200 & 50 & 6 & 3 & 1.000 

## DGP1, 2, 3 Alternative

In [208]:
def get_performance_metrics_alt(true, estimate_file, dgp_file, bootstrap_iterations=200, g_true_index=-1):
    with open(dgp_file, "rb") as g:
        with open(estimate_file, "rb") as f:
            data = pickle.load(g)
            estimates = pickle.load(f)
            results = []
            for i, (e, d) in enumerate(zip(estimates, data)):
                # display(e)
                beta = e[0]
                beta_analytical_se = e[2]
                conf_beta = e[1]

                # Get bias
                g_true = d[g_true_index]
                bias = np.mean(beta - true[g_true])
                # Get RMSE
                mse = np.mean((beta - true[g_true]) ** 2)
                # Get confidence size
                conf_size = (conf_beta[:, :, 1] - conf_beta[:, :, 0]).mean()
                # Get coverage
                coverage = np.mean((true[g_true] >= conf_beta[:, :, 0]) & (true[g_true] <= conf_beta[:, :, 1]))
                # Get mean se
                mean_se = np.mean(beta_analytical_se)
                results.append(
                    {
                        "bias": bias,
                        "mse": mse,
                        "conf_size": conf_size,
                        "coverage": coverage,
                        "mean_se": mean_se,
                    }
                )
            results_df = pd.DataFrame(results)
            results_means = results_df.mean()
            results_means["rmse"] = np.sqrt(results_means["mse"])
            results_means.drop(columns=["mse"], inplace=True)
            return results_means.to_dict()

In [209]:
for dgp in (1, 2, 3):
    results = []
    for n in (100, 200):
        for t in (20, 50):
            for g in (3, 6):
                for k in (3,):
                    estimate_file = f"estimates/dgp{dgp}_n{n}_t{t}_G{g}_k{k}_ols.pkl"
                    dgp_file = f"development/generated_data/dgp{dgp}_n{n}_t{t}_G{g}_k{k}.pkl"
                    true = np.array([[i, i, i] for i in range(1, g + 1)])

                    g_true_index = -1 if dgp == 1 else 3 if dgp == 2 else 2

                    metrics = get_performance_metrics_alt(true, estimate_file, dgp_file, g_true_index=g_true_index)

                    # Convert dict to DataFrame (assume keys are metric names, values are scalars or 1D)
                    row = dict(metrics)  # in case it's already a dict
                    row["n"] = n
                    row["t"] = t
                    row["G"] = g
                    row["k"] = k

                    results.append(row)

    # Now convert the list of dicts into a proper DataFrame
    final_results = pd.DataFrame(results)

    # Optional: reorder columns
    cols = ["n", "t", "G", "k"] + [col for col in final_results.columns if col not in ["n", "t", "G", "k"]]
    final_results = final_results[cols]
    print(f"DGP {dgp}")
    print(
        final_results.round(3)
        .sort_values(by="G")
        .to_latex(float_format="%.3f", index=False, escape=False, column_format="l" + "r" * (len(cols) - 1))
    )

DGP 1
\begin{tabular}{lrrrrrrrrr}
\toprule
n & t & G & k & bias & mse & conf_size & coverage & mean_se & rmse \\
\midrule
100 & 20 & 3 & 3 & 0.001 & 0.181 & 1.727 & 0.950 & 0.408 & 0.425 \\
100 & 50 & 3 & 3 & -0.001 & 0.072 & 1.062 & 0.952 & 0.264 & 0.269 \\
200 & 20 & 3 & 3 & -0.001 & 0.186 & 1.751 & 0.950 & 0.413 & 0.431 \\
200 & 50 & 3 & 3 & 0.001 & 0.073 & 1.068 & 0.950 & 0.265 & 0.270 \\
100 & 20 & 6 & 3 & -0.001 & 0.180 & 1.730 & 0.952 & 0.408 & 0.425 \\
100 & 50 & 6 & 3 & 0.000 & 0.073 & 1.064 & 0.949 & 0.264 & 0.270 \\
200 & 20 & 6 & 3 & 0.002 & 0.185 & 1.747 & 0.950 & 0.412 & 0.430 \\
200 & 50 & 6 & 3 & -0.000 & 0.074 & 1.071 & 0.950 & 0.266 & 0.271 \\
\bottomrule
\end{tabular}

DGP 2
\begin{tabular}{lrrrrrrrrr}
\toprule
n & t & G & k & bias & mse & conf_size & coverage & mean_se & rmse \\
\midrule
100 & 20 & 3 & 3 & 0.001 & 0.875 & 3.619 & 0.950 & 0.854 & 0.936 \\
100 & 50 & 3 & 3 & -0.001 & 0.312 & 2.102 & 0.951 & 0.522 & 0.559 \\
200 & 20 & 3 & 3 & -0.001 & 0.886 & 3.633 & 